# 06. Functions and Side Effects

This section is basically a warning. A warning on mixing mutable data and functions. It can be very useful but, more frequent than not, it make the program harder to reason about.

Lets see this with examples. Consider the following function:

In [1]:
seen_items = []

def have_i_seen_the_item(item):
    result = item in seen_items
    
    if not result:
        seen_items.append(item)
        
    return result

In the example above we have a function that tracks if we have seen a value and returns whether we've seen it. Cool, it seems to do what it is meant to:

In [2]:
print(have_i_seen_the_item(1))
print(have_i_seen_the_item(1))

False
True


What is the problem with it you may ask? Well, you now have a **function that behaves differently for the same input** depending on an opaque state (opaque in the sense that it is not obvious the function has a state by looking at its signature). More over, you may have other functions that depend on such state and that don't look trivially connected. This is what we call a side effect:

In [3]:
def print_not_seen_item(item):
    if not have_i_seen_the_item(item):
        print(item)

        
for item in [1, 2, 3, 4, 5]:
    if not have_i_seen_the_item(item):
        print_not_seen_item(item)

Here we have a function that is named `print_not_seen_item(item)` the name tells us that that is the function we want to call when we want to print a not seen item. So that is what we do. The fact that it makes sure we are not trying to print a seen item is an implementation detail but because `have_i_seen_the_item(item)` has a hidden state it becomes relevant how the later behaves to understand the behaviour of the first one. This is the risk of having side effects.

Now imagine having to understand all the implementation details for all your functions in your program of three thousand lines of code divided across 20 files. Madness! It becomes really hard to extend your program.

Ok, fine, hidden mutable state is bad. How can we do better?

Consider the example bellow, it implements the same behaviour as the functions above but there is no hidden state. In fact we make it explicit:

In [4]:
def new_have_i_seen_the_item(item, seen_items):
    return item in seen_items


def new_print_not_seen_item(item, seen_items):
    if not new_have_i_seen_the_item(item, seen_items):
        print(item)
        
def update_seen_items(item, seen_items):
    seen_items.append(item)

new_seen_items = []

for item in [1, 2, 3, 4, 5]:
    if not new_have_i_seen_the_item(item, new_seen_items):
        new_print_not_seen_item(item, new_seen_items)
        update_seen_items(item, new_seen_items)

1
2
3
4
5


The main difference in the example above is that the state is explicitly stated and, more importantly, **the state** is not changed as an extra functionality of a function but it **is explicitly updated in a function that states that it does so**.